In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [3]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [4]:
def kron(A, B):
    return la.kron(A, B)

In [5]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [5]:
print(kronsum(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

In [6]:
n = 1
p_num = 100
eps_G = 10 ** (-8)
eps_Phi = 10 ** (-6)
eps_p_i = 10 ** (-6)

In [8]:
# Входной BMAP
matrD_0 = np.array([[-4.69791416]])
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

for k in range(1, n+1):
    matrD_k.append(-matrD_0)

for matr in matrD_k:
    print(matr)

[[-4.69791416]]
[[ 4.69791416]]


In [9]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
matr_a = copy.deepcopy(matrD_1_)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)

matrdD_1_ = copy.deepcopy(matrD_k[1])

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = theta.dot(matrdD_1_).dot(vect_e)
print(lamD)

[ 4.69791416]


In [10]:
# Поток поломок MAP
matrH0 = np.array([[-6.00065225]])
matrH1 = np.array([[6.00065225]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = gamma.dot(matrH1).dot(vect_e)
print(h)

[[ 0.00000000]]
[[ 1.00000000]]
[ 6.00065225]


In [11]:
# Поток обслуживания PH1
# TODO make e vectors dynamically allocated!
beta1 = np.matrix([1])
matrS1 = np.matrix([[-10]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1]])
matrS1_0 = - matrS1 * M1_e
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
mu_1 = -la.inv(beta1.dot(la.inv(matrS1)).dot(vect_e))
print(mu_1)

[[ 10.00000000]]


In [12]:
# Поток обслуживания PH2
beta2 = np.matrix([1])
matrS2 = np.matrix([[-1]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1]])
matrS2_0 = - matrS2 * M2_e

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(beta2.dot(la.inv(matrS2)).dot(vect_e))
print(mu_2)

[[ 1.00000000]]


In [13]:
matrS_w = kron(matrS1_0 * beta1, M2_e * beta2) + kron(M1_e * beta1, matrS2_0 * beta2)

In [14]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([1])
matrA1 = np.matrix([[-29.99985287]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1]])
matrA1_0 = - matrA1.dot(L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(alpha1.dot(la.inv(matrA1)).dot(vect_e))
print(kappa_1)

[[ 29.99985287]]


In [14]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([1])
matrA2 = np.matrix([[-89.99955862]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1]])
matrA2_0 = - matrA2.dot(L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(alpha2.dot(la.inv(matrA2)).dot(vect_e))
print(kappa_2)

In [16]:
# Поток ремонта PH
tau = np.matrix([1])
matrT = np.matrix([[-100.0098]])
T_e = np.matrix([[1]])
matrT0 = - matrT.dot(T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -la.inv(tau.dot(la.inv(matrT)).dot(vect_e))
print(phi)

[[ 100.00980000]]


In [16]:
a = W_ * V_

In [18]:
# Q~0 строится правильно
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0)

4
[[-10.69856641  0.00000000  0.00000000  6.00065225]
 [ 89.99955862 -100.69812503  6.00065225  0.00000000]
 [ 0.00000000  100.00980000 -104.70771416  0.00000000]
 [ 100.00980000  0.00000000  29.99985287 -134.70756703]]


In [31]:
# Q~k строится правильно
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
pd.DataFrame(matrQw_k[1])
for matr in matrQw_k:
    print(matr)
sum = 0
for i in range(matrQw_k[0].shape[0]):
    for matr in matrQw_k:
        sum += np.sum(matr[i])
print('matrQw_k[' + str(i) + '] = ', sum)

[[-10.69856641  0.00000000  0.00000000  6.00065225]
 [ 89.99955862 -100.69812503  6.00065225  0.00000000]
 [ 0.00000000  100.00980000 -104.70771416  0.00000000]
 [ 100.00980000  0.00000000  29.99985287 -134.70756703]]
[[ 4.69791416  0.00000000  0.00000000  0.00000000]
 [ 0.00000000  4.69791416  0.00000000  0.00000000]
 [ 0.00000000  0.00000000  4.69791416  0.00000000]
 [ 0.00000000  0.00000000  0.00000000  4.69791416]]
matrQw_k[3] =  -5.3290705182e-15


In [20]:
# Q^0 строится правильно
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQv_0)

,0,1,2,3
0,10.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0


In [21]:
# Q_0 строится правильно
block0 = kron(np.eye(a), matrS1_0.dot(beta1))
block1 = kron(kron(np.eye(a), matrS2_0.dot(beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0.dot(beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQ_0)

,0,1,2,3
0,10.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0


In [32]:
# Q_1 строится правильно
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])
pd.DataFrame(matrQ_1)

,0,1,2,3
0,-20.698566,0.000000,0.000000,6.000652
1,89.999559,-101.698125,6.000652,0.000000
2,0.000000,100.009800,-105.707714,0.000000
3,100.009800,0.000000,29.999853,-134.707567


In [35]:
#Q_k - считалась на одну меньше чем надо - исправлено, правильно
matrQ_k = [matrQ_0, matrQ_1] 
for k in range(2, n+2):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

# print(matrQ_k[2][0])
# sum = 0

for matr in matrQ_k:
    print(matr)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for j in range(1, np.array(matrQ_k).shape[0]):
        sum += np.sum(matrQ_k[j][i])
    print('matrQ_k[' + str(i) + '] = ', sum)

[[ 10.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00000000  1.00000000  0.00000000  0.00000000]
 [ 0.00000000  0.00000000  1.00000000  0.00000000]
 [ 0.00000000  0.00000000  0.00000000  0.00000000]]
[[-20.69856641  0.00000000  0.00000000  6.00065225]
 [ 89.99955862 -101.69812503  6.00065225  0.00000000]
 [ 0.00000000  100.00980000 -105.70771416  0.00000000]
 [ 100.00980000  0.00000000  29.99985287 -134.70756703]]
[[ 4.69791416  0.00000000  0.00000000  0.00000000]
 [ 0.00000000  4.69791416  0.00000000  0.00000000]
 [ 0.00000000  0.00000000  4.69791416  0.00000000]
 [ 0.00000000  0.00000000  0.00000000  4.69791416]]
matrQ_k[0] =  1.7763568394e-15
matrQ_k[1] =  1.7763568394e-15
matrQ_k[2] =  3.5527136788e-15
matrQ_k[3] =  -1.06581410364e-14


In [25]:
# Проверка генератора Q

In [27]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])

def calc_G(matrG_prev):
    temp_sum = matrQ_k[0].copy()
    for k in range(1, n+1):
        temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG_prev, k))
    matrG_new = matrQ_1_neg_inv.dot(temp_sum)
    return matrG_new
        
matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)
i = 1
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    i += 1
    print(i,  'norm = ', la.norm(matrG - matrG_old, ord=np.inf))

pd.DataFrame(matrG)

2 norm =  2.6302541801314145
3 norm =  2.6302541801314154
4 norm =  2.630254180131415
5 norm =  2.630254180131415
6 norm =  2.6302541801314154
7 norm =  2.6302541801314154
8 norm =  2.6302541801314163
9 norm =  2.6302541801314163
10 norm =  2.6302541801314168
11 norm =  2.6302541801314163
12 norm =  2.6302541801314163
13 norm =  2.6302541801314163
14 norm =  2.6302541801314163
15 norm =  2.6302541801314163
16 norm =  2.6302541801314168
17 norm =  2.630254180131417
18 norm =  2.630254180131417
19 norm =  2.6302541801314177
20 norm =  2.630254180131418
21 norm =  2.630254180131418
22 norm =  2.630254180131418
23 norm =  2.6302541801314185
24 norm =  2.6302541801314194
25 norm =  2.63025418013142
26 norm =  2.63025418013142
27 norm =  2.6302541801314203
28 norm =  2.6302541801314208
29 norm =  2.6302541801314208
30 norm =  2.6302541801314216
31 norm =  2.6302541801314216
32 norm =  2.6302541801314216
33 norm =  2.6302541801314216
34 norm =  2.6302541801314216
35 norm =  2.6302541801314216

2.630254180131469
301 norm =  2.6302541801314696
302 norm =  2.630254180131469
303 norm =  2.630254180131469
304 norm =  2.6302541801314696
305 norm =  2.6302541801314696
306 norm =  2.6302541801314696
307 norm =  2.6302541801314705
308 norm =  2.630254180131471
309 norm =  2.630254180131471
310 norm =  2.6302541801314714
311 norm =  2.6302541801314714
312 norm =  2.630254180131472
313 norm =  2.6302541801314723
314 norm =  2.6302541801314723
315 norm =  2.6302541801314727
316 norm =  2.630254180131473
317 norm =  2.6302541801314736
318 norm =  2.6302541801314745
319 norm =  2.630254180131475
320 norm =  2.6302541801314754
321 norm =  2.630254180131475
322 norm =  2.6302541801314754
323 norm =  2.630254180131476
324 norm =  2.630254180131476
325 norm =  2.630254180131476
326 norm =  2.630254180131476
327 norm =  2.630254180131476
328 norm =  2.630254180131476
329 norm =  2.630254180131476
330 norm =  2.6302541801314767
331 norm =  2.6302541801314767
332 norm =  2.6302541801314763
333 n

2.630254180131514
631 norm =  2.6302541801315145
632 norm =  2.630254180131515
633 norm =  2.6302541801315145
634 norm =  2.6302541801315145
635 norm =  2.630254180131515
636 norm =  2.6302541801315154
637 norm =  2.6302541801315154
638 norm =  2.630254180131516
639 norm =  2.630254180131516
640 norm =  2.6302541801315162
641 norm =  2.6302541801315167
642 norm =  2.6302541801315167
643 norm =  2.630254180131516
644 norm =  2.6302541801315162
645 norm =  2.6302541801315167
646 norm =  2.6302541801315167
647 norm =  2.6302541801315167
648 norm =  2.6302541801315167
649 norm =  2.630254180131517
650 norm =  2.6302541801315176
651 norm =  2.6302541801315176
652 norm =  2.630254180131517
653 norm =  2.6302541801315176
654 norm =  2.6302541801315176
655 norm =  2.630254180131518
656 norm =  2.6302541801315185
657 norm =  2.630254180131519
658 norm =  2.6302541801315193
659 norm =  2.6302541801315193
660 norm =  2.6302541801315193
661 norm =  2.63025418013152
662 norm =  2.6302541801315202
6

2.6302541801316366
1106 norm =  2.6302541801316366
1107 norm =  2.6302541801316366
1108 norm =  2.6302541801316366
1109 norm =  2.630254180131637
1110 norm =  2.6302541801316375
1111 norm =  2.6302541801316375
1112 norm =  2.6302541801316375
1113 norm =  2.6302541801316375
1114 norm =  2.6302541801316375
1115 norm =  2.6302541801316375
1116 norm =  2.6302541801316375
1117 norm =  2.630254180131638
1118 norm =  2.630254180131638
1119 norm =  2.6302541801316384
1120 norm =  2.6302541801316384
1121 norm =  2.6302541801316393
1122 norm =  2.6302541801316397
1123 norm =  2.6302541801316397
1124 norm =  2.63025418013164
1125 norm =  2.63025418013164
1126 norm =  2.6302541801316406
1127 norm =  2.6302541801316406
1128 norm =  2.630254180131641
1129 norm =  2.6302541801316415
1130 norm =  2.630254180131642
1131 norm =  2.630254180131643
1132 norm =  2.6302541801316424
1133 norm =  2.6302541801316424
1134 norm =  2.6302541801316424
1135 norm =  2.6302541801316424
1136 norm =  2.6302541801316424

 2.6302541801317902
1779 norm =  2.6302541801317902
1780 norm =  2.6302541801317902
1781 norm =  2.6302541801317902
1782 norm =  2.6302541801317902
1783 norm =  2.6302541801317907
1784 norm =  2.630254180131791
1785 norm =  2.630254180131791
1786 norm =  2.630254180131791
1787 norm =  2.630254180131791
1788 norm =  2.6302541801317916
1789 norm =  2.630254180131792
1790 norm =  2.630254180131792
1791 norm =  2.6302541801317925
1792 norm =  2.630254180131793
1793 norm =  2.630254180131793
1794 norm =  2.630254180131793
1795 norm =  2.6302541801317934
1796 norm =  2.630254180131794
1797 norm =  2.630254180131794
1798 norm =  2.6302541801317947
1799 norm =  2.630254180131795
1800 norm =  2.630254180131795
1801 norm =  2.6302541801317956
1802 norm =  2.6302541801317956
1803 norm =  2.6302541801317956
1804 norm =  2.630254180131796
1805 norm =  2.6302541801317965
1806 norm =  2.6302541801317965
1807 norm =  2.6302541801317965
1808 norm =  2.6302541801317973
1809 norm =  2.6302541801317973
18

...

norm =  2.63025418013859
59751 norm =  2.63025418013859
59752 norm =  2.63025418013859
59753 norm =  2.63025418013859
59754 norm =  2.63025418013859
59755 norm =  2.63025418013859
59756 norm =  2.63025418013859
59757 norm =  2.63025418013859
59758 norm =  2.6302541801385906
59759 norm =  2.6302541801385906
59760 norm =  2.6302541801385906
59761 norm =  2.6302541801385906
59762 norm =  2.6302541801385906
59763 norm =  2.6302541801385906
59764 norm =  2.630254180138591
59765 norm =  2.630254180138591
59766 norm =  2.6302541801385915
59767 norm =  2.630254180138591
59768 norm =  2.6302541801385915
59769 norm =  2.6302541801385915
59770 norm =  2.6302541801385915
59771 norm =  2.630254180138592
59772 norm =  2.630254180138592
59773 norm =  2.630254180138592
59774 norm =  2.630254180138592
59775 norm =  2.6302541801385924
59776 norm =  2.6302541801385924
59777 norm =  2.6302541801385932
59778 norm =  2.6302541801385937
59779 norm =  2.6302541801385937
59780 norm =  2.6302541801385937
59781 

2.630254180138629
60108 norm =  2.630254180138629
60109 norm =  2.6302541801386297
60110 norm =  2.63025418013863
60111 norm =  2.63025418013863
60112 norm =  2.63025418013863
60113 norm =  2.63025418013863
60114 norm =  2.6302541801386305
60115 norm =  2.6302541801386305
60116 norm =  2.630254180138631
60117 norm =  2.630254180138631
60118 norm =  2.630254180138631
60119 norm =  2.630254180138631
60120 norm =  2.6302541801386314
60121 norm =  2.630254180138632
60122 norm =  2.630254180138632
60123 norm =  2.630254180138632
60124 norm =  2.630254180138632
60125 norm =  2.630254180138632
60126 norm =  2.630254180138632
60127 norm =  2.6302541801386323
60128 norm =  2.6302541801386323
60129 norm =  2.6302541801386323
60130 norm =  2.6302541801386323
60131 norm =  2.6302541801386328
60132 norm =  2.6302541801386337
60133 norm =  2.6302541801386337
60134 norm =  2.6302541801386337
60135 norm =  2.6302541801386337
60136 norm =  2.6302541801386337
60137 norm =  2.630254180138634
60138 norm =

60543 norm =  2.6302541801386825
60544 norm =  2.6302541801386825
60545 norm =  2.6302541801386825
60546 norm =  2.630254180138682
60547 norm =  2.630254180138682
60548 norm =  2.6302541801386825
60549 norm =  2.630254180138682
60550 norm =  2.6302541801386825
60551 norm =  2.6302541801386825
60552 norm =  2.6302541801386825
60553 norm =  2.6302541801386825
60554 norm =  2.630254180138683
60555 norm =  2.6302541801386834
60556 norm =  2.6302541801386834
60557 norm =  2.6302541801386834
60558 norm =  2.6302541801386834
60559 norm =  2.6302541801386834
60560 norm =  2.6302541801386834
60561 norm =  2.6302541801386834
60562 norm =  2.630254180138684
60563 norm =  2.630254180138684
60564 norm =  2.6302541801386843
60565 norm =  2.6302541801386843
60566 norm =  2.6302541801386843
60567 norm =  2.6302541801386847
60568 norm =  2.6302541801386847
60569 norm =  2.6302541801386847
60570 norm =  2.630254180138685
60571 norm =  2.630254180138685
60572 norm =  2.630254180138685
60573 norm =  2.630


60967 norm =  2.630254180138727
60968 norm =  2.6302541801387274
60969 norm =  2.630254180138728
60970 norm =  2.630254180138728
60971 norm =  2.630254180138728
60972 norm =  2.630254180138728
60973 norm =  2.630254180138728
60974 norm =  2.6302541801387282
60975 norm =  2.6302541801387282
60976 norm =  2.6302541801387282
60977 norm =  2.6302541801387282
60978 norm =  2.6302541801387282
60979 norm =  2.6302541801387287
60980 norm =  2.6302541801387287
60981 norm =  2.6302541801387287
60982 norm =  2.6302541801387287
60983 norm =  2.6302541801387287
60984 norm =  2.6302541801387287
60985 norm =  2.6302541801387287
60986 norm =  2.630254180138729
60987 norm =  2.630254180138729
60988 norm =  2.630254180138729
60989 norm =  2.630254180138729
60990 norm =  2.6302541801387296
60991 norm =  2.6302541801387296
60992 norm =  2.6302541801387296
60993 norm =  2.6302541801387296
60994 norm =  2.63025418013873
60995 norm =  2.6302541801387305
60996 norm =  2.6302541801387305
60997 norm =  2.63025

KeyboardInterrupt: 

In [ ]:
for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    print('Sum matrQw_k[' + str(i) + '] = ', sum)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for matr in matrQ_k:
        sum += np.sum(matr[i])
    print('Sum matrQ_k[' + str(i) + '] = ', sum)

In [ ]:
temp_sum = matrQ_k[1].copy()
for k in range(2, n+1):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = (-matrG_0).dot(matrQv_0)
pd.DataFrame(matrG_0)

In [ ]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = matrQw_k[l].copy()
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k].copy()
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n+1, p_num):
            matrQ_il[i].append(np.zeros(matrQ_il[i][l % n+1].shape)) 
    else:
        for l in range(0, p_num):
            if l >= i and l - i <= n:
                temp_matr = matrQ_k[l-i].copy()
                for k in range(l+1, n+1):
                    if (k - i <= 3):
                        mult_matr = matrQ_k[k-i].copy()
                        for kk in range(k-1, l-1, -1):
                            if kk == 0:
                                mult_matr = mult_matr.dot(matrG_0)
                            else:
                                mult_matr = mult_matr.dot(matrG)
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))
    
print(len(matrQ_il[3]))
pd.DataFrame(la.inv(-matrQ_il[1][1]))

In [ ]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = matrPhi_l[0].dot(matrQ_il[0][l]).dot(la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
print(la.norm(matrPhi_l[25], np.inf))

In [ ]:
matr_a = np.transpose(- matrQ_il[0][0])
matr_b = np.zeros((matr_a.shape[0], 1))
p0 = la.lstsq(matr_a, matr_b)[3]
p0 = np.transpose(p0)

print(type(a), type(R_))

vect_e1 = np.array([[1.] for i in range(0, p0.shape[0])])
print(vect_e1)
temp_vect = vect_e1.copy()
for l in range(1, p_num):
    vect_e2 = np.array([[1.] for i in range(0, matrPhi_l[l].shape[1])])
    temp_vect += matrPhi_l[l].dot(vect_e2)
res = p0.dot(temp_vect)[0]
print('temp_vect = ', temp_vect)
p0 /= res
print(p0.sum())

In [ ]:
vect_p_l = [p0]
for l in range(1, n+1):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(vect_p_l[l])

In [ ]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

In [ ]:
matr_a = np.transpose(matrGamma)
matr_b = np.zeros((matr_a.shape[0], 1))
x = la.lstsq(matr_a, matr_b)[3]
vect_e = np.array([[1.] for i in range(0, x.shape[0])])
res = x.dot(vect_e)[0]
x /= res
print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

In [ ]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

In [ ]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

In [ ]:
print(la.norm(np.eye(matrQ_il[0][0].shape[0]).dot(matrQ_il[0][1]).dot(la.inv(-matrQ_il[1][1])), np.inf))

In [ ]:
result = np.sum(matrQv_0[0])
i = 0
for matrQk in matrQ_k:
    if i > 0:
        result += np.sum(matrQk[0])
    i += 1
print(result)